In [2]:
from efficientnet.keras import EfficientNetB5
import glob
import os

#hyperparams
width = 200
height = 200
input_shape = (width, height, 3)

trainpath = '~/Downloads/Courses/HCSDLDPT/data/train'
testpath = '~/Downloads/Courses/HCSDLDPT/data/test'
valpath = '~/Downloads/Courses/HCSDLDPT/data/dev'


pretrained_conv = EfficientNetB5(weights='imagenet', include_top=False, input_shape=input_shape)

Using TensorFlow backend.
/home/vudat1710/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/vudat1710/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/vudat1710/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/vudat1710/.local/lib/python3.7/site-packages/tenso

Instructions for updating:
Colocations handled automatically by placer.


KeyboardInterrupt: 